# xDB Training

A notebook exploring the xDB dataset tier3 part for Building Segmentation Model and Damaged Classification Model

In [187]:
# In a Jupyter notebook or IPython environment, run this in the first cell
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [188]:
import sys
import os
sys.path.append(os.path.abspath('../src'))

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
# Define the transformation pipeline
def get_train_augmentation_pipeline(image_size=(256, 256)):
    transform = A.Compose([
        # Resize images and masks
        A.Resize(image_size[0], image_size[1], p=1.0),  # Ensure both image and mask are resized
        # Random horizontal flip
        A.HorizontalFlip(p=0.5),
        # Random vertical flip
        A.VerticalFlip(p=0.5),
        # Random rotation
        A.RandomRotate90(p=0.5),
        # Random brightness and contrast adjustments
        A.RandomBrightnessContrast(p=0.2),
        # Random contrast adjustment
        A.RandomGamma(p=0.2),
        # Random scale and aspect ratio change
        A.RandomSizedCrop(min_max_height=(image_size[0], image_size[1]), height=image_size[0], width=image_size[1], p=0.5),
        # Random blur
        A.GaussianBlur(p=0.2),
        # Random saturation adjustment
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.2),
        # Random crop (if you want to focus on specific regions)
        A.CenterCrop(height=image_size[0], width=image_size[1], p=1.0),
        # Convert to tensor (works for both image and mask)
        ToTensorV2()
    ])
    return transform

def get_val_augmentation_pipeline(image_size=(256, 256)):
    transform = A.Compose([
        # Resize images and masks
        A.Resize(image_size[0], image_size[1], p=1.0),  # Ensure both image and mask are resized
        # Random horizontal flip
        A.HorizontalFlip(p=0.5),
        # Random vertical flip
        A.VerticalFlip(p=0.5),
        # Random rotation
        A.RandomRotate90(p=0.5),
        # Random contrast adjustment
        ToTensorV2()
    ])
    return transform

In [ ]:
from datasets import xDB_Damaged_Building

data_train  = xDB_Damaged_Building(
    origin_dir = "../data/xDB/tier3",
    mode="building",
    time="pre",
    transform=None,
    type="train"
)


Loaded 5097 train labels.
